# Лабораторная № 1. Свертка тензорной сети.

## **Задание:**
Выполнить сретку тензорной сети из 5 тензоров. Свертку тензоров нужно выполнить 3-мя способами:
1. С помощью инструментов **ncon** library.
2. С помощью инструментов **numpy** library.
3. С помощью нативной реализации на Python.

Замерить время решения и выполнить сравнение подходов к реализации.

In [9]:
import numpy as np
from ncon import ncon as nc

Инциализируем случайные тензоры

In [59]:
i0 = 1
i1 = 2
i2 = 3
i3 = 4

j = 5
m = 6
h = 7

a = 8
b = 8
c = 8


A = np.random.uniform(-1, 1, (a, j, i0))
B = np.random.uniform(-1, 1, (j, c, m, i1))
C = np.random.uniform(-1, 1, (i0, i1, i2, i3))
D = np.random.uniform(-1, 1, (b, i2, h))
E = np.random.uniform(-1, 1, (h, i3, m))

print(f'Tensor A shape: {A.shape }')
print(f'Tensor B shape: {B.shape }')
print(f'Tensor C shape: {C.shape }')
print(f'Tensor D shape: {D.shape }')
print(f'Tensor E shape: {E.shape }')

Tensor A shape: (8, 5, 1)
Tensor B shape: (5, 8, 6, 2)
Tensor C shape: (1, 2, 3, 4)
Tensor D shape: (8, 3, 7)
Tensor E shape: (7, 4, 6)


## Ncon implementation

Самый простой способ реализации. Готовое решение из "коробки". Минимум кода, производительность высокая.

In [60]:
def contract_network_ncon(A, B, C, D, E):
    return nc(
        (A, B, C, D, E), 
        ( 
            [-1,  2,  1    ], # [a,  j,  i0    ]
            [ 2, -2,  5,  3], # [j,  c,  m,  i1]
            [ 1,  3,  6,  4], # [i0, i1, i2, i3]
            [-3,  6,  7    ], # [b,  i2, h     ]
            [ 7,  4,  5    ]  # [h,  i3, m     ]
         )
    ) 

In [61]:
%%timeit
result_ncon = contract_network_ncon(A, B, C, D, E)

242 µs ± 5.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [62]:
result_ncon = contract_network_ncon(A, B, C, D, E)

print(f'result_ncon.shape = {result_ncon.shape}')
print(result_ncon)

result_ncon.shape = (8, 8, 8)
[[[ 1.55154289e+00 -3.79267827e-01 -5.16796701e-01  2.00425218e+00
    1.10963699e+00  2.44095286e+00  2.84039486e+00 -1.53805650e+00]
  [ 1.32403600e+00  6.66613851e-01  1.28485134e-01  4.80502391e-01
    1.36132810e+00  4.06270431e+00  3.66822542e-01 -2.77057886e+00]
  [ 3.52221714e-01  1.21086240e+00 -1.68384052e+00  1.03804134e+00
    1.31017434e+00  7.97295115e-01  9.45355651e-01 -1.39028790e+00]
  [-9.60029131e-01  2.51902393e+00 -2.66460855e+00  5.99457378e-01
    1.86412001e+00 -1.82898876e+00 -7.01228715e-02 -1.38343508e-01]
  [ 4.50053342e-01  1.29693260e-01 -1.11207084e+00  1.08220752e+00
   -3.44260146e+00 -2.31478137e+00 -4.39362844e-01 -2.81340662e-01]
  [-1.08910986e+00 -8.34920854e-01  3.15024654e+00 -3.27054477e+00
    2.07460633e-01 -5.48444497e-01 -3.06996424e+00  1.34078809e+00]
  [-1.11992046e+00  4.08675652e-01  2.07287874e+00 -1.02830840e+00
    6.54882257e-01 -2.19653630e-01 -1.28512447e+00  6.35280385e-01]
  [ 3.63690176e-01  1.131

## Numpy implementation

Данный способ сложнее с точки зрения кода и его поддержки, однако этот способ реализации выигрывает по времени исполнения.

In [63]:
def contract_network_numpy(A, B, C, D, E, with_debug_info=False):
    
    AC_1 = np.tensordot(A, C, axes=([2], [0]))
    ACB_1 = np.tensordot(AC_1,B, axes=([1, 2], [0, -1]))
    ACBE_1 = np.tensordot(ACB_1, E, axes=([2, -1], [-2, -1]))
    result = np.tensordot(ACBE_1, D, axes=([1, -1], [-2, -1]))
    
    if not with_debug_info:
        return result
    
    print("Tensor AC_1 shape:", AC_1.shape)
    print("Tensor ACB_1 shape:", ACB_1.shape)
    print("Tensor ACBE_1 shape:", ACBE_1.shape)
    print("Tensor result shape:", result.shape)
    
    return result

In [64]:
%%timeit
result_np = contract_network(A, B, C, D, E)

112 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [65]:
print("Numpy implementation result:")
result_np = contract_network(A, B, C, D, E, with_debug_info=True)


print("Difference between Numpy and NCON implementations:")
print(result_ncon - result_np)

Numpy implementation result:
Tensor AC_1 shape: (8, 5, 2, 3, 4)
Tensor ACB_1 shape: (8, 3, 4, 8, 6)
Tensor ACBE_1 shape: (8, 3, 8, 7)
Tensor result shape: (8, 8, 8)
Difference between Numpy and NCON implementations:
[[[-2.22044605e-16  1.11022302e-16 -2.22044605e-16  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [-2.22044605e-16 -2.22044605e-16  4.99600361e-16  2.77555756e-16
   -6.66133815e-16  0.00000000e+00  1.11022302e-16  0.00000000e+00]
  [-5.55111512e-17  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00 -1.11022302e-16  1.11022302e-16  0.00000000e+00]
  [-1.11022302e-16 -4.44089210e-16  0.00000000e+00 -1.11022302e-16
   -2.22044605e-16 -2.22044605e-16  6.93889390e-17  0.00000000e+00]
  [-2.77555756e-16 -6.10622664e-16  0.00000000e+00  2.22044605e-16
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.44089210e-16
   -1.66533454e-16  0.00000000e+00  0.0000

## Native implementation

Самый дорогой и неоправданный способ реализации. Работает медленно, реализация слишком сложная и минимально гибкая

In [68]:
def contract_network_native(A, B, C, D, R):
    AC_1 = np.zeros((a, j, i1, i2, i3))
    for a_i in range(a):
        for j_i in range(j):
            for i1_i in range(i1):
                for i2_i in range(i2):
                    for i3_i in range(i3):
                        for i0_i in range(i0):
                            ac_idx = a_i, j_i, i1_i, i2_i, i3_i
                            AC_1[ac_idx] = AC_1[ac_idx] + A[a_i][j_i][i0_i] * C[i0_i][i1_i][i2_i][i3_i]

    ACB_1 = np.zeros((a, i2, i3, c, m))
    for a_i in range(a):
        for i2_i in range(i2):
            for i3_i in range(i3):
                for c_i in range(c):
                    for m_i in range(m):
                        for j_i in range(j):
                            for i1_i in range(i1):  
                                acb_idx = a_i, i2_i, i3_i, c_i, m_i
                                ACB_1[acb_idx] = ACB_1[acb_idx] + AC_1[a_i, j_i, i1_i, i2_i, i3_i] * B[j_i, c_i, m_i, i1_i]


    ACBE_1 = np.zeros((a, i2, c, h))
    for a_i in range(a):
        for i2_i in range(i2):
            for c_i in range(c):
                for h_i in range(h):
                    for i3_i in range(i3):
                        for m_i in range(m):
                            abce_idx = a_i, i2_i, c_i, h_i
                            acb_idx = a_i, i2_i, i3_i, c_i, m_i
                            ACBE_1[abce_idx] = ACBE_1[abce_idx] + ACB_1[acb_idx] * E[h_i, i3_i, m_i]
        
    result  = np.zeros((a, c, b))
    for a_i in range(a):
        for c_i in range(c):
            for b_i in range(b):
                for i2_i in range(i2):
                    for h_i in range(h):
                        result[a_i, c_i, b_i] =  result[a_i, c_i, b_i] + ACBE_1[a_i, i2_i, c_i, h_i] + D[b_i, i2_i, h_i]
    
    return result

In [69]:
%%timeit   
result_native = contract_network_native(A, B, C, D, E)

84.8 ms ± 558 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [70]:
result_native = contract_network_native(A, B, C, D, E)
print("Native result shape:", result_native.shape)
print(result_native)

Native result shape: (8, 8, 8)
[[[ 2.45326976e+00  1.52835337e+00 -1.07628465e+00  2.54930982e+00
    4.53601799e+00  6.26816783e+00  5.28310893e-01 -4.16115174e+00]
  [ 1.41056021e+00  4.85643810e-01 -2.11899421e+00  1.50660027e+00
    3.49330843e+00  5.22545828e+00 -5.14398663e-01 -5.20386130e+00]
  [ 6.23872863e+00  5.31381224e+00  2.70917422e+00  6.33476869e+00
    8.32147686e+00  1.00536267e+01  4.31376976e+00 -3.75692871e-01]
  [ 8.84040365e+00  7.91548725e+00  5.31084923e+00  8.93644371e+00
    1.09231519e+01  1.26553017e+01  6.91544478e+00  2.22598214e+00]
  [ 1.34428863e+00  4.19372236e-01 -2.18526578e+00  1.44032869e+00
    3.42703686e+00  5.15918670e+00 -5.80670236e-01 -5.27013287e+00]
  [-2.28776544e+00 -3.21268184e+00 -5.81731986e+00 -2.19172539e+00
   -2.05017218e-01  1.52713263e+00 -4.21272431e+00 -8.90218695e+00]
  [ 4.49936923e-01 -4.74979475e-01 -3.07961749e+00  5.45976981e-01
    2.53268515e+00  4.26483499e+00 -1.47502195e+00 -6.16448458e+00]
  [ 3.28926002e+00  2.36